In [3]:

import torch
from torch_geometric.data import Data,Batch,Dataset,InMemoryDataset
from loader import BioDataset,DblpDataset,ChemDatasetInmemory
import networkx as nx
import copy
import numpy as np
from copy import deepcopy
from torch_geometric.utils import to_networkx,k_hop_subgraph,subgraph,dropout_adj
import os.path as osp

In [10]:
dataset=ChemDatasetInmemory('../data/chem/toxcast/',data_type='toxcast')
data=dataset.get(0)
for key in data.keys:
    print(key)
temp={key:data[key].numpy().tolist() for key in data.keys}
# print(temp)
import json
with open('data.json', 'w') as fw:
    json.dump(temp,fw)
with open('data.json','r') as f:
    abc = json.load(f)
import json
for i in range(len(dataset)):
    data=dataset.get(i)
    temp={key:data[key].numpy().tolist() for key in data.keys}
    with open('../data/chem/toxcast/raw/graph_'+str(i)+'.json', 'w') as fw:
        json.dump(temp,fw)

8576
id
edge_index
y
x
edge_attr


In [7]:
dataset[0]

Data(x=[10, 2], edge_index=[2, 20], edge_attr=[20, 2], y=[617], id=[1])

In [ ]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

In [ ]:
data_list = [data, data]
edge_index = Batch.from_data_list(data_list).edge_index
batch=Batch.from_data_list(data_list)

In [ ]:
'''
Author: your name
Date: 2022-03-13 16:53:32
LastEditTime: 2022-03-30 16:17:31
LastEditors: your name
Description: 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
FilePath: /wb/MGE/code/test.ipynb
'''
data, slices = InMemoryDataset.collate(data_list)
torch.save((data, slices), './a.pt')

In [ ]:
data1=torch.load('./a.pt')
data1[0]

In [3]:

root_unsupervised = '../data/bio/unsupervised'
dataset = BioDataset(root_unsupervised, data_type='unsupervised')
temp=dataset.get(0)

1


In [ ]:
from torch_geometric.data import Data
from util import graphTransform
from loader_copy1 import DblpDataset1
root_unsupervised = '../data/dblp/unsupervised'
dataset = DblpDataset1(root_unsupervised,'unsupervised',True,pre_transform=graphTransform())
# # dir_list=['/graph_'+str(i)+'.json' for i in range(754862)]
# for i in range(754862):
#     dir=raw_path+str(i//10000)+'/graph_'+str(i)+'.json'
#     with open(dir,'r') as f:
#         temp = json.load(f)
#     data = Data(x=torch.tensor(temp['x'],dtype=torch.float32),
#                                     edge_index=torch.tensor(temp['edge_index'],dtype=torch.int64),
#                                     edge_attr=torch.tensor(temp['edge_attr'],dtype=torch.float32),
#                                     center_node_idx=torch.tensor(temp['center_node_idx'],dtype=torch.int64),
#                                     species_id=torch.tensor(temp['species_id'],dtype=torch.int64),
#                                     go_target_downstream=torch.tensor(temp['go_target_downstream'],dtype=torch.int64)
#                                     )

Processing...


KeyboardInterrupt: 

In [3]:
root_unsupervised = '../data/dblp/unsupervised'
dataset = DblpDataset(root_unsupervised, data_type='unsupervised')

In [5]:
temp=dataset.get(0)
temp

Data(center_node_idx=[1], edge_attr=[230, 1], edge_index=[2, 230], go_target_downstream=[25], species_id=[1], x=[96, 4])

In [6]:

def node_preprocess(data):

    node_num,_=data.x.size()

    G=to_networkx(data)
    
    degree_nodes=nx.degree_centrality(G)
    # eigenvector_nodes=nx.eigenvector_centrality(G)
    closeness_nodes=nx.closeness_centrality(G)
    betweenness_nodes=nx.betweenness_centrality(G)
    
    degree_center_node=max(degree_nodes.keys(), key=(lambda k: degree_nodes[k]))
    # eigenvector_center_node=max(eigenvector_nodes.keys(), key=(lambda k: eigenvector_nodes[k]))
    closeness_center_node=max(closeness_nodes.keys(), key=(lambda k: closeness_nodes[k]))
    betweenness_center_node=max(betweenness_nodes.keys(), key=(lambda k: betweenness_nodes[k]))
    
    k_sub_degree=k_hop_subgraph(degree_center_node,1,data.edge_index)
    # k_sub2=k_hop_subgraph(eigenvector_center_node,1,data.edge_index)
    k_sub_closeness=k_hop_subgraph(closeness_center_node,1,data.edge_index)
    k_sub_betweenness=k_hop_subgraph(betweenness_center_node,1,data.edge_index)
    
    # temp=sorted(zip(degree_nodes.values(),degree_nodes.keys()),reverse=True)
    # k_sub_temp1=k_hop_subgraph(temp[0][1],1,data.edge_index)
    # k_sub_temp2=k_hop_subgraph(temp[1][1],1,data.edge_index)
    # k_sub_temp3=k_hop_subgraph(temp[2][1],1,data.edge_index)
    data1=copy.copy(data)
    data2=copy.copy(data)
    data3=copy.copy(data)
    
    data1.edge_index,data1.edge_attr=subgraph(k_sub_degree[0],data.edge_index,data.edge_attr,num_nodes=node_num)
    data2.edge_index,data2.edge_attr=subgraph(k_sub_closeness[0],data.edge_index,data.edge_attr,num_nodes=node_num)
    data3.edge_index,data3.edge_attr=subgraph(k_sub_betweenness[0],data.edge_index,data.edge_attr,num_nodes=node_num)
    return data1,data2,data3

In [7]:
def graph_preprocess(data):
    node_num,_=data.x.size()
    drop_num = int(node_num / 10)
    idx_drop = np.random.choice(node_num, drop_num, replace=False)
    idx_nondrop = [n for n in range(node_num) if not n in idx_drop]
    
    sub_num = int(node_num * 0.2)
    sub_drop = np.random.choice(node_num, sub_num, replace=False)
    sub_nondrop = [n for n in range(node_num) if not n in sub_drop]
    
    data1=copy.copy(data)
    data2=copy.copy(data)
    data3=copy.copy(data)
    
    data1.edge_index,data1.edge_attr=subgraph(idx_nondrop,data.edge_index,data.edge_attr, num_nodes=node_num)
    
    data2.edge_index,data2.edge_attr=dropout_adj(data.edge_index,data.edge_attr,p=0.2)
    data3.edge_index,data3.edge_attr=subgraph(sub_nondrop,data.edge_index,data.edge_attr,num_nodes=node_num)


    return data1,data2,data3

In [ ]:
{key: for key in data.keys:}

In [15]:
torch.save(data,'temp.pt')

RuntimeError: [enforce fail at inline_container.cc:298] . unexpected pos 3631995904 vs 3631995824

In [8]:
data=dataset.get(0)
for key in data.keys:
    print(key)
temp={key:data[key].numpy().tolist() for key in data.keys}
# print(temp)
import json
with open('data.json', 'w') as fw:
    json.dump(temp,fw)
with open('data.json','r') as f:
    abc = json.load(f)

id
edge_index
y
x
edge_attr


In [9]:
abc

{'id': [0],
 'edge_index': [[0, 1, 1, 2, 1, 3, 3, 4, 4, 5, 5, 6, 6, 7, 6, 8, 8, 9, 9, 3],
  [1, 0, 2, 1, 3, 1, 4, 3, 5, 4, 6, 5, 7, 6, 8, 6, 9, 8, 3, 9]],
 'y': [-1.0,
  -1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -1.0,
  -

In [37]:
import json
for i in range(len(dataset)):
    data=dataset.get(i)
    temp={key:data[key].numpy().tolist() for key in data.keys}
    with open('../data/bio/unsupervised/raw/graph_'+str(i)+'.json', 'w') as fw:
        json.dump(temp,fw)
    # if i>100:
    #     break


In [2]:
dataset2 = BioDataset('../data/bio/supervised', data_type='supervised')

1000


/home/qingmeiwang/anaconda3/envs/py3.8/lib/python3.8/site-packages/torch_geometric/data/dataset.py:150: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, make sure to sure to delete '../data/bio/supervised/processed' first
  warnings.warn(


In [39]:
dataset2.get(0)

Data(center_node_idx=[1], edge_attr=[316, 9], edge_index=[2, 316], go_target_downstream=[40], go_target_pretrain=[5000], species_id=[1], x=[67, 1])

In [42]:
for i in range(len(dataset3)):
    data=dataset3.get(i)
    temp={key:data[key].numpy().tolist() for key in data.keys}
    with open('../data/bio/supervised/raw/graph_'+str(i)+'.json', 'w') as fw:
        json.dump(temp,fw)
    # if i>100:
    #     break

In [2]:
from loader_copy import DblpDataset
dataset3 = DblpDataset('../data/dblp/supervised', data_type='supervised')

In [6]:
import json
for i in range(len(dataset3)):
    data=dataset3.get(i)
    temp={key:data[key].numpy().tolist() for key in data.keys}
    with open('../data/dblp/unsupervised/raw/graph_'+str(i)+'.json', 'w') as fw:
        json.dump(temp,fw)
    # if i>100:
    #     break

In [7]:
with open('../data/dblp/unsupervised/raw/graph_0.json','r') as f:
    abc = json.load(f)

In [10]:
abc.keys()

dict_keys(['x', 'edge_index', 'edge_attr', 'center_node_idx', 'species_id', 'go_target_downstream'])

In [4]:
a=[{key:dataset3.get(i)[key].numpy().tolist() for key in dataset3.get(i).keys} for i in range(10000)]

In [ ]:
del a[:]

In [45]:
len(a)

10

In [ ]:
from multiprocessing import cpu_count
from multiprocessing import Pool
n=int(len(data_list)/8)  #data_getsubchar[i*n:len(data_getsubchar)]
p=Pool(8)
for i in range(8):
    if i!=7:
        p.apply_async(batch_preprocess,args=(data_list[i*n:(i+1)*n],i))
    else:
        p.apply_async(batch_preprocess,args=(data_list[i*n:len(data_list)],i))
p.close()
p.join()

In [35]:
with open('../data/bio/unsupervised/raw/graph_0.json','r') as f:
    abc = json.load(f)

In [36]:
abc

{'x': [[1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0],
  [1.0]],
 'edge_index': [[0,
   13,
   0,
   49,
   0,
   21,
   1,
   13,
   1,
   9,
   1,
   22,
   2,
   27,
   2,
   12,
   2,
   16,
   2,
   3,
   2,
   39,
   3,
   27,
   3,
   24,
   3,
   23,
   3,
   32,
   3,
   40,
   3,
   18,
   3,
   14,
   4,
   7,
   4,
   50,
   4,
   23,
   4,
   40,
   4,
   18,
   4,
   14,
   5,
   23,
   5,
   32,
   5,
   40,
   5,
   18,
   6,
   42,
   6,
   30,
   6,
   10,
   6,
   33,
   7,
   50,
   7,
   14,
   8,
   33,
   8,
   10,
   8,
   30,
   9,
   13,
   9,
   15,
   10,
   37,
   

In [32]:
data.center_node_idx.dtype

torch.int64

In [27]:
for key in abc.keys:
    print(key)

TypeError: 'builtin_function_or_method' object is not iterable

In [26]:
torch.tensor(abc['x'],dtype=torch.float32)
torch.tensor(abc['edge_index'],dtype=torch.int64)

tensor([[ 0, 13,  0, 49,  0, 21,  1, 13,  1,  9,  1, 22,  2, 27,  2, 12,  2, 16,
          2,  3,  2, 39,  3, 27,  3, 24,  3, 23,  3, 32,  3, 40,  3, 18,  3, 14,
          4,  7,  4, 50,  4, 23,  4, 40,  4, 18,  4, 14,  5, 23,  5, 32,  5, 40,
          5, 18,  6, 42,  6, 30,  6, 10,  6, 33,  7, 50,  7, 14,  8, 33,  8, 10,
          8, 30,  9, 13,  9, 15, 10, 37, 10, 42, 10, 47, 10, 30, 10, 33, 11, 13,
         12, 16, 12, 39, 13, 27, 13, 31, 13, 35, 13, 20, 13, 22, 13, 49, 13, 21,
         14, 27, 14, 50, 14, 23, 14, 40, 14, 18, 14, 17, 14, 41, 14, 46, 14, 26,
         15, 43, 15, 35, 15, 28, 15, 20, 15, 29, 15, 36, 16, 27, 16, 24, 16, 17,
         16, 39, 17, 27, 17, 50, 17, 41, 17, 22, 18, 27, 19, 34, 19, 44, 19, 45,
         20, 36, 20, 28, 20, 29, 21, 49, 22, 41, 23, 27, 23, 32, 25, 34, 25, 44,
         25, 45, 26, 46, 27, 44, 27, 43, 27, 40, 27, 35, 27, 30, 28, 43, 29, 43,
         30, 37, 30, 42, 30, 47, 30, 33, 33, 37, 33, 42, 34, 45, 34, 44, 35, 43,
         36, 43, 38, 43, 41,

In [9]:
f = open('temp.txt','w')
f.write(str(temp))
f.close()
f = open('temp.txt','r')
a = f.read()
dict_name = eval(a)
f.close()

In [14]:
a

"{'x': [[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0]], 'edge_index': [[0, 13, 0, 49, 0, 21, 1, 13, 1, 9, 1, 22, 2, 27, 2, 12, 2, 16, 2, 3, 2, 39, 3, 27, 3, 24, 3, 23, 3, 32, 3, 40, 3, 18, 3, 14, 4, 7, 4, 50, 4, 23, 4, 40, 4, 18, 4, 14, 5, 23, 5, 32, 5, 40, 5, 18, 6, 42, 6, 30, 6, 10, 6, 33, 7, 50, 7, 14, 8, 33, 8, 10, 8, 30, 9, 13, 9, 15, 10, 37, 10, 42, 10, 47, 10, 30, 10, 33, 11, 13, 12, 16, 12, 39, 13, 27, 13, 31, 13, 35, 13, 20, 13, 22, 13, 49, 13, 21, 14, 27, 14, 50, 14, 23, 14, 40, 14, 18, 14, 17, 14, 41, 14, 46, 14, 26, 15, 43, 15, 35, 15, 28, 15, 20, 15, 29, 15, 36, 16, 27, 16, 24, 16, 17, 16, 39, 17, 27, 17, 50, 17, 41, 17, 22, 18, 27, 19, 34, 19, 44, 19, 45, 20, 36, 20, 

In [12]:
# torch.tensor(np.array(a['x']),dtype=torch.float())
b=torch.from_numpy(np.array(a['x']))
# graph=Data(x=a['x'])

TypeError: string indices must be integers

In [8]:
dataset[0].x.shape

torch.Size([51, 1])

In [ ]:
for i in range(len(dataset)):
    data=dataset.get(i)
    for key in data.keys:
        data[key]
    # data, slices = InMemoryDataset.collate(data_list)
    # torch.save((data, slices), osp.join('../data/bio/unsupervised/processed/', f'data_{i}.pt'))
    torch.save(Batch.from_data_list(data_list),'../data/dblp/unsupervised/processed/data_'+str(i)+'.pt')
    if i>1000:
        break

In [8]:
# dataset.get(1)

# len(dataset)


for i in range(len(dataset)):
    data=dataset.get(i)
    data1,data2,data3=node_preprocess(data)
    data4,data5,data6=graph_preprocess(data)
    data_list=[data,data1,data2,data3,data4,data5,data6]
    # data, slices = InMemoryDataset.collate(data_list)
    # torch.save((data, slices), osp.join('../data/bio/unsupervised/processed/', f'data_{i}.pt'))
    torch.save(Batch.from_data_list(data_list),'../data/dblp/unsupervised/processed/data_'+str(i)+'.pt')
    if i>1000:
        break

In [ ]:
import os
len(os.listdir('../data/bio/unsupervised/processed/'))

In [8]:
# data,slices=torch.load('../data/bio/unsupervised/processed/data_0.pt')
batch=torch.load('../data/bio/unsupervised/processed/data_0.pt')

In [26]:

temp=[]
for i in range(len(batch)):
    temp.append(batch[i])

In [27]:
a1,a2,a3,a4,a5,a6,a7=temp

In [28]:
a1

Data(center_node_idx=[1], edge_attr=[228, 9], edge_index=[2, 228], species_id=[1], x=[51, 1])

In [18]:
a1=batch[0]

In [24]:
batch

Batch(batch=[357], center_node_idx=[7], edge_attr=[876, 9], edge_index=[2, 876], ptr=[8], species_id=[7], x=[357, 1])

In [23]:
a1

('batch',
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 

In [14]:
print(batch[0])
print(batch[1])
print(batch[6])
len(batch)

Data(center_node_idx=[1], edge_attr=[228, 9], edge_index=[2, 228], species_id=[1], x=[51, 1])
Data(center_node_idx=[1], edge_attr=[52, 9], edge_index=[2, 52], species_id=[1], x=[51, 1])
Data(center_node_idx=[1], edge_attr=[154, 9], edge_index=[2, 154], species_id=[1], x=[51, 1])


7

In [13]:
torch.load(osp.join('../data/bio/unsupervised/processed/', f'data_{0}.pt'))

Batch(batch=[357], center_node_idx=[7], edge_attr=[876, 9], edge_index=[2, 876], ptr=[8], species_id=[7], x=[357, 1])

In [ ]:
data

In [ ]:
data.keys

In [ ]:
slices

In [ ]:
from itertools import repeat

In [ ]:
data_=copy.copy(data)
data_.[key]=data.keys

In [ ]:
print(data)
print(data.keys)
print(slices)
data['x'][0:10]

In [ ]:

for i in range(len(slices)-1):
    data_=copy.copy(data)
    start,end=slices[i],slices[i+1]
    for key in data.keys:
        data_[key]=data[key][start:end]
    data_list.append(data_)
# data= Data()

In [ ]:
del data_list[:]

In [ ]:
data_list

In [ ]:
aaa=data_list[0]
aaa.keys

In [ ]:
a.keys

In [ ]:
data1[0]

In [ ]:
edge_index
batch.batch

In [ ]:
torch.save(batch,'./a.pt')
a=torch.load('./a.pt')

In [2]:
# -*- coding: utf-8 -*-
import time
from multiprocessing.dummy import Pool as ThreadPool
def process(item):
   print('正在并行for循环')
   print(item)
   time.sleep(5)
items = ['apple', 'bananan', 'cake', 'dumpling']
pool = ThreadPool()
pool.map(process, items)
pool.close()
pool.join()



正在并行for循环
apple
正在并行for循环
bananan
正在并行for循环
cake
正在并行for循环
dumpling
